In [1]:
from matplotlib import pyplot as plt
from IPython import display
import numpy as np
import random
from matplotlib import colors
from __future__ import division
import copy

%matplotlib inline

In [2]:
test = 1*np.ones((30,30))
T = 1000
test_boltz = {"4.0" : np.exp(-8/T), "2.0" : np.exp(-4/T)}
init = test[:,:]
random.seed(1)

In [3]:
N = 30
init = np.reshape([random.choice([1, -1]) for k in range(N*N)], (N,N))

In [4]:
X = np.shape(init)[1]
Y = np.shape(init)[0]
N = X * Y

# treats the 2D array into a 1D array ("flattened") to find the neighboring indexes 
# neighbors are directly above, below, or next to the point and wraps around
# format: 
#   {index: [neighbor to the "left", neighbor to the "right", neighbor "below", neighbor "above"]}
neighboring_iis = {i : [(i//X)*X + (i-1)%X, (i//X)*X + (i+1)%X,
                        (i+X)%N, (i-X)%N] for i in range(N)}

grid = init[:]
n = 1000
for i in range(n): # iterating over steps
    s = np.random.randint(N) # choose a random spin s in the grid
    # look at its neighbors in neighboring_iis, and check that its spins are the same as s
    # if its spins are the same, add to the cluster with probability 1-np.exp(-2*J/T)
    
    # after above process is done, flip the spins in the cluster
    # by multiplying those iis in the grid by -1
    
    # plot the updated wolff grid